In [9]:
import gograph
import importlib
import re
import time
import annotations

import pandas as pd
import numpy as np

In [2]:
%%time

importlib.reload(annotations)

fp = 'data/goa_human.gaf'

anno = annotations.Annotations(fp)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 1.67 s, sys: 191 ms, total: 1.86 s
Wall time: 2.48 s


In [18]:
srs = anno.annotations.groupby('GO_ID').size()

In [22]:
list(srs.values)

[12,
 4,
 2,
 14,
 14,
 4,
 1,
 6,
 3,
 2,
 3,
 3,
 29,
 23,
 28,
 1,
 6,
 2,
 2,
 16,
 89,
 7,
 78,
 23,
 5,
 2,
 6,
 10,
 8,
 23,
 5,
 33,
 12,
 40,
 58,
 2,
 112,
 24,
 143,
 4,
 6,
 1,
 6,
 6,
 3,
 2,
 9,
 5,
 1,
 1,
 1,
 48,
 2,
 1,
 1105,
 33,
 24,
 5,
 7,
 12,
 40,
 30,
 10,
 1107,
 23,
 27,
 6,
 76,
 2,
 129,
 2,
 2,
 4,
 36,
 1,
 14,
 265,
 61,
 2,
 3,
 9,
 1,
 47,
 23,
 15,
 25,
 8,
 5,
 21,
 165,
 15,
 49,
 142,
 33,
 305,
 2,
 7,
 3,
 4,
 3,
 1,
 4,
 3,
 3,
 2,
 142,
 32,
 5,
 23,
 5,
 18,
 28,
 2,
 2,
 4,
 3,
 4,
 2,
 8,
 5,
 1,
 1,
 31,
 4,
 2,
 1,
 6,
 10,
 145,
 64,
 1,
 237,
 13,
 29,
 12,
 2,
 3,
 2,
 2,
 1,
 4,
 2,
 14,
 5,
 1,
 1,
 1,
 39,
 4,
 4,
 2,
 2,
 1,
 8,
 4,
 13,
 12,
 23,
 25,
 6,
 2,
 3,
 2,
 25,
 3,
 25,
 104,
 2,
 3,
 74,
 1,
 6,
 3,
 7,
 527,
 29,
 8,
 1,
 10,
 2,
 30,
 13,
 53,
 1,
 1,
 1,
 1,
 53,
 55,
 9,
 2,
 5,
 1,
 3,
 8,
 5,
 11,
 8,
 2,
 3,
 1,
 3,
 3,
 19,
 1,
 29,
 17,
 4,
 8,
 3,
 17,
 3,
 3,
 6,
 1,
 12,
 4,
 2,
 2,
 65,
 14,
 7,
 3,
 5,
 2

In [3]:
%%time
importlib.reload(gograph)
ontology = gograph.OBOParser('data/go-basic.obo').parse_ontology()

CPU times: user 1.71 s, sys: 62.1 ms, total: 1.78 s
Wall time: 1.79 s


In [4]:
ontology.draw_connections()

In [5]:
t = time.time()
ontology.get_ancestry_table()
print(time.time() - t)

7.841043949127197


In [25]:
importlib.reload(gograph)
am = gograph.AncestryMatrix(ontology.full_ancestry)

In [26]:
am.get_matrix()

In [53]:
am.matrix.todense()

MemoryError: 

In [13]:
from scipy import sparse

In [17]:
np.ones(5)

array([1., 1., 1., 1., 1.])

In [59]:
n = 10000
a = list(range(n))

mm = sparse.coo_matrix((np.ones(n), (np.array(a), np.array(a))))

mm = sparse.coo_matrix((np.ones(n), (a,a)))

mm.todense()

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]])

In [52]:
pd.DataFrame(mm.todense(), index=mm.row, columns=mm.col)

,0,1,2,3,4
0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0


In [29]:
len(ontology.full_ancestry)

47434

In [38]:
i = 0
for k, term in ontology.nodes.items():
    if term.ancestor_ids == []:
        #print(term.id)
        i += 1
print(i)

2858


In [37]:
ontology.nodes['GO:0000001'].__dict__

{'id': 'GO:0000001',
 'namespace': 'biological_process',
 'definition': '"The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton." [GOC:mcc, PMID:10873824, PMID:11389764]',
 'is_a': ['GO:0048308 ! organelle inheritance',
  'GO:0048311 ! mitochondrion distribution'],
 'ancestor_ids': ['GO:0048308', 'GO:0048311'],
 'child_ids': [],
 'ancestors': [<gograph.GoTerm at 0x7f9f4dcc2e48>,
 'children': [],
 'name': 'mitochondrion inheritance'}

In [38]:
ontology.get_full_ancestry('GO:0000001')

['GO:0009987',
 'GO:0051179',
 'GO:0051640',
 'GO:0008150',
 'GO:0051646',
 'GO:0048308',
 'GO:0051641',
 'GO:0048311',
 'GO:0007005',
 'GO:0071840',
 'GO:0006996',
 'GO:0016043']

In [9]:
%%time 
importlib.reload(gograph)

obo_fp = 'data/go-basic.obo'
obo_file = open(obo_fp, 'r')

def parse_ancestor_id(is_a):
    """
    Extracts go term ids from 'is_a' and 'relatioship' field
    """
    capture = re.search('(GO:\d{7})\s!\s', is_a)
    if capture:
        return capture.group(1)

go_list = {}
blank_template = {'id': None, 'name': None, 'namespace': None,
                  'def': None, 'is_a': []}

go_dag = gograph.GoGraph()
term_flag = False
for line in obo_file:
    capture = re.search('(.+):\s(.+)\n', line)
    if line == '[Term]\n':
        term_flag = True
    elif line == '\n':
        term_flag = False
        
    if capture and term_flag:
        name = capture.group(1)
        content = capture.group(2)
        if name == 'id':
            go_term = gograph.GoTerm(id_=content)
            go_dag.add_node(go_term) # push term into tree
        elif name == 'is_a' or name == 'relationship':
            go_term.is_a.append(content)
            go_term.ancestor_ids.append(parse_ancestor_id(content))
        elif name == 'name':
            go_term.name = content
        elif name == 'namespace':
            go_term.namespace = content
        elif name == 'def':
            go_term.definition = content

obo_file.close()

CPU times: user 1.9 s, sys: 15.6 ms, total: 1.91 s
Wall time: 1.91 s


In [ ]:
go_dag.nodes['GO:0000015'].__dict__

In [ ]:
go_dag.draw_connections()

In [ ]:
t0 = time.time()
h = {}

i = 0
for node in go_dag.nodes.values():
    anc = traverse3(node)
    h[node.id] = list(set(anc))
    #i = i + 1
    #if i % 1000 == 0 or i == len(go_dag.nodes):
    #   bar = '*' * (i//1000)
    #    print(bar, end='\r')
        #print(f'Progress {i}/{len(go_dag.nodes)} done', end='\r')
print()
print('Wall time: ', time.time()-t0)
#['GO:0000001'].ancestors

In [ ]:
t0 = time.time()
h = {}

i = 0
for node in go_dag.nodes.keys():
    anc = traverse4(node)
    h[node] = list(set(anc))
    #i = i + 1
    #if i % 1000 == 0 or i == len(go_dag.nodes):
    #   bar = '*' * (i//1000)
    #    print(bar, end='\r')
        #print(f'Progress {i}/{len(go_dag.nodes)} done', end='\r')
print()
print('Wall time: ', time.time()-t0)
#['GO:0000001'].ancestors

In [ ]:
t0 = time.time()
def traverse2(term):
    all_ancestors = []
    if term.ancestors == []:
        all_ancestors.append(term.id)
    else:
        all_ancestors.append(term.id)
        for t in term.ancestors:
            all_ancestors.extend(traverse2(t))
    return all_ancestors
        
    
a = traverse2(go_dag.nodes['GO:0000001'])
print(time.time()-t0)
print(a)
print(len(a))
print(len(set(a)))

In [ ]:
t0 = time.time()
def traverse3(term):
    all_ancestors = [term.id]
    if term.ancestors != []:
        for t in term.ancestors:
            all_ancestors.extend(traverse3(t))
    return all_ancestors
        
b = traverse3(go_dag.nodes['GO:0000001'])
print(time.time()-t0)

print(b)
print(len(b))
print(len(set(b)))

In [ ]:
def traverse4(node_id):
    """
    Traverse GO graph from given node to root

    Attributes:
        node_id : str
            GO term id of node (ex: 'GO:0000001')
    Return:
        ancestors : list
            list of acestor GO term ids

    Note:
        The list of ancestor ids will have duplicates, because
        in the GO ontology children can have multiple parents and
        so there are multiple paths leading to the root.

        For example:

                /C\
            A--B   F--root
                \D/

        Calling traverse(A) will return
            [A, B, C, F, root, D, F, root]

    """
    ancestors = []

    #f go_dag.nodes[node_id].ancestors == []:
    #   return ancestors
    if go_dag.nodes[node_id].ancestors != []:
        for ancestor in go_dag.nodes[node_id].ancestors:
            ancestors.append(ancestor.id)
            ancestors.extend(traverse4(ancestor.id))

    return ancestors

t0 = time.time()
b = traverse4('GO:0000001')
print(time.time()-t0)
print(b)
print(len(b))
print(len(set(b)))

In [ ]:
from progress.bar import Bar

In [ ]:
bar = Bar('Processing', max=20)
for i in range(20):
    # Do some work
    bar.next()
bar.finish()

In [ ]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    
    author : https://stackoverflow.com/a/34325723
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
import time

# A List of Items
items = list(range(0, 57))
l = len(items)

# Initial call to print 0% progress
printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
for i, item in enumerate(items):
    # Do stuff...
    time.sleep(0.1)
    # Update Progress Bar
    printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

In [ ]:
10/5

In [ ]:
1+1

In [ ]:
import numpy as np

In [ ]:
np.array([[1,1],[1,1]]) * np.array([[10],[2]])

In [ ]:
2+2